In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import update_display, display, Markdown
import anthropic

In [2]:
import gradio as gr

In [3]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCp


In [4]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [5]:
system_prompt = "You are a helpful assistant"

In [6]:
def message_gpt(prompt):
    response = openai.chat.completions.create(
        messages = [
            {'role':'system','content':system_prompt},
            {'role':'user','content':prompt}
                   ],
        model = 'gpt-4o-mini'
    )
    result = response.choices[0].message.content
    return result

In [7]:
message_gpt("How many colors in rainbow are there?")

'There are seven colors in a rainbow. They are commonly listed as red, orange, yellow, green, blue, indigo, and violet. These colors can be remembered by the acronym ROYGBIV.'

# User Interface

In [8]:
def shout(text):
    print(f"The text printed is :{text}")
    return text.upper()

In [9]:
shout('longitude')

The text printed is :longitude


'LONGITUDE'

In [10]:
gr.Interface(fn=shout, inputs='textbox',outputs='textbox',allow_flagging ='never').launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://942d4166466ee25e64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The text printed is :hexo


In [11]:
gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label='Your Message:',lines = 6)],
    outputs=[gr.Textbox(label='Response:',lines = 8)],
    allow_flagging='never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://435cccee950c318548.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# OpenAI chat interface

In [12]:
gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label='Your Message:',lines = 6)],
    outputs=[gr.Textbox(label='Response:',lines = 8)],
    allow_flagging='never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://a6511e969ef09667ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Results in Markdown and formatted format

In [13]:
gr.Interface(
    fn=message_gpt,
    inputs = [gr.Textbox(label ='Your Message:',lines=8)],
    outputs= [gr.Markdown(label ='Response:')],
    allow_flagging='never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://23eb8cc15fb786ad43.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Stream the results

In [14]:
def stream_gpt(prompt):
    stream = openai.chat.completions.create(
        messages = [
            {'role':'user','content':prompt},
            {'role':'system','content':system_prompt}
        ],
        model = 'gpt-4o-mini',
        stream = True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```','').replace('markdown','')
        yield response

In [15]:
gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label = 'Your Message:',lines = 10)],
    outputs=[gr.Markdown(label = 'Response:')],
    allow_flagging='never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://0f3874b89d394389e4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Bad Solution if we do stream the chunk by chunk

In [16]:
def stream_gpt_chunk(prompt):
    stream = openai.chat.completions.create(
        messages = [
            {'role':'user','content':prompt},
            {'role':'system','content':system_prompt}
        ],
        model = 'gpt-4o-mini',
        stream = True
    )

    for chunk in stream:  
        yield chunk.choices[0].delta.content or ''

In [17]:
gr.Interface(
    fn=stream_gpt_chunk,
    inputs=[gr.Textbox(label = 'Your Message:',lines = 10)],
    outputs=[gr.Markdown(label = 'Response:')],
    allow_flagging='never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://a267209fb7e1a8a198.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Anthropic Chat Interface (Streaming)

In [19]:
def stream_claude(prompt):
    response = claude.messages.stream(
        model ='claude-3-haiku-20240307',
        max_tokens = 400,
        messages = [{'role':'user','content':prompt}],
        system=system_prompt
    )
    result = ''
    with response as stream:
        for text in stream.text_stream:
            result += text or ''
            yield result

In [20]:
gr.Interface(
    fn = stream_claude,
    inputs = [gr.Textbox(label = 'Your Message :',lines = 5)],
    outputs = [gr.Markdown(label = 'Response :')],
    allow_flagging = 'never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://9daf871d067e476452.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Gemini API Chat Interface

In [21]:
def stream_google(prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.5-flash-lite',
        system_instruction=system_prompt
    )

    result = gemini.generate_content(prompt, stream=True)

    response = ''
    for chunk in result:
        if chunk.text:               
            response += chunk.text
            yield response

In [22]:
gr.Interface(
    fn = stream_google,
    inputs=[gr.Textbox(label='Your Message:',lines = 5)],
    outputs = [gr.Markdown(label ='Response:')],
    allow_flagging = 'never'
).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:414: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://131e1d2a76df6f7ab6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
def stream_model(prompt,model):
    if model == 'GPT':
        result = stream_gpt(prompt)
    elif model =='Claude':
        result = stream_claude(prompt)
    elif model == 'Gemini':
        result = stream_google(prompt)
    else:
        raise ValueError("Unknown Model")
    yield from result

In [24]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude","Gemini"], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://37ba4d0fb324ddf3f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
